# Attention is All You Need ! 🙂


The transformers approach to this.

Attention is a mechanism that allows models to focus on the most relevant parts of the input sequence when making predictions. Instead of processing all words with equal importance, the model assigns weights to different words based on their relevance to the task.

In the context of movie review sentiment analysis, attention helps the model identify crucial words (like "excellent" or "terrible") and gives them more importance, even if they appear far apart in the sentence. This improves the model’s ability to capture key sentiment cues from long or complex reviews.


Let's try to use BERT

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Adding the required imports

In [ ]:
import os
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
    TrainerCallback
)
from datasets import Dataset
import wandb

## Setting up WandB to monitor our model's loss in realtime

In [ ]:
os.environ["WANDB_DISABLED"] = "false"
wandb.init(project="bert-sentiment-analysis")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Checking if you have a GPU or CPU
I'm going to be using the T4 run time on Colab but if you decide to use the normal one, you can do that as well.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Loading the dataset

I am going to load the dataset and perform label encoding on it and then do the train-test split

In [ ]:
# Load dataset
train_df = pd.read_csv('IMDB Dataset.csv')
train_df['sentiment'] = train_df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['review'].tolist(),
    train_df['sentiment'].tolist(),
    test_size=0.2,
    random_state=42
)

### Tokenization

So we split the dataset into training and validation sets. It uses the BERT tokenizer to prepare the data by truncating and padding reviews to a fixed length for consistent input rather than making our tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=512)

In [ ]:
train_data = Dataset.from_dict({'text': train_texts, 'label': train_labels})
val_data = Dataset.from_dict({'text': val_texts, 'label': val_labels})

In [ ]:
train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
val_data = val_data.map(tokenize, batched=True, batch_size=len(val_data))

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

## Defining Training Arguments for BERT

Here I am defining all the training arguments that we will need to use for training BERT. The comments describe what each argument does

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',  # Directory to save model checkpoints
    evaluation_strategy="epoch",  # Evaluate after every epoch
    save_strategy="epoch",  # Save checkpoints at the end of each epoch
    learning_rate=2e-5,  # Reduced learning rate for BERT fine-tuning
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=1e-4,
    logging_dir='./logs',  # Directory for logs
    logging_steps=10,
    save_total_limit=1,  # Only keep the latest checkpoint
    load_best_model_at_end=True,  # Restore best model based on validation
    report_to="wandb"  # Enable wandb logging
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Loding BERT
We are loading BERT from the Transformers library and we are using the base uncased model so that it will not differentiate between uppercase and lowercase letters.

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## Modifications to BERT

Let's make a few modifications on BERT. We will unfreeze the last two encoder layers, which can help the model retain BERT's language understanding while fine-tuning on the new dataset and also unfreeze the fully connected layers.

In [ ]:
for param in model.bert.encoder.layer[-2:].parameters():
    param.requires_grad = True

In [ ]:
for param in model.bert.encoder.layer[:-2].parameters():
    param.requires_grad = False

## Training BERT

Here I am going to use the HuggingFace trainer class to train BERT using the training aruguments that we defined earlier.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,0.272100,0.227991
2,0.172000,0.200357
3,0.122500,0.213096


TrainOutput(global_step=7500, training_loss=0.19578389985958736, metrics={'train_runtime': 10094.4319, 'train_samples_per_second': 11.888, 'train_steps_per_second': 0.743, 'total_flos': 3.15733266432e+16, 'train_loss': 0.19578389985958736, 'epoch': 3.0})

In [ ]:
for key, value in results.items():
    print(f"{key}: {value:.4f}")

eval_loss: 0.2004
eval_runtime: 313.4228
eval_samples_per_second: 31.9060
eval_steps_per_second: 1.9940
epoch: 3.0000



## Interpretation
The evaluation loss of 0.2004 indicates the model's error on the validation set, suggesting it generalizes well with low error. With an evaluation runtime of 313.42 seconds, the model processed about 32 samples per second, showing reasonable efficiency.

In [ ]:
wandb.finish()

eval/loss,█▁▄▁
eval/runtime,▄▁▃█
eval/samples_per_second,▅█▆▁
eval/steps_per_second,▅█▆▁
train/epoch,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇██
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█
train/grad_norm,▂▂▄▃▄▃█▃▄▃▃▁▂▃▃▁▃▃▃▂▅▅▃▄▂▃▄▁▃▃▃▃▃▃▄▃▄▁▄▁
train/learning_rate,████▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁
train/loss,▇█▆▇█▅▅▅▇▅▅▇▆▅▄▅▃█▄▅▄▅▅▆▄▄▅▃▁▅▆▁▄▃▅▅▅█▇▅
eval/loss,0.20036
eval/runtime,313.4228


In [ ]:
# I will use additional metrics
from sklearn.metrics import precision_recall_fscore_support


## Additional Metrics
I also want to see other metrics like precision, accuracy, recall and F1 just so that I can compare it with the other LSTM models

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    preds = predictions.argmax(axis=-1)  # logits to predicted class

    # Calculating precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    # Calculating accuracy
    accuracy = (preds == labels).mean()

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics  # Passing the custom metric function
)


## Evaluation re run

In [ ]:
# Evaluate the model
results = trainer.evaluate()

# Print the evaluation results
print(f"Validation Accuracy: {results['eval_accuracy'] * 100:.2f}%")
print(f"Validation Precision: {results['eval_precision'] * 100:.2f}%")
print(f"Validation Recall: {results['eval_recall'] * 100:.2f}%")
print(f"Validation F1 Score: {results['eval_f1'] * 100:.2f}%")

wandb: Currently logged in as: karanlvm123 (karanlvm123-ut-arlington-uta-the-university-of-texas-at-). Use `wandb login --relogin` to force relogin


Validation Accuracy: 92.99%
Validation Precision: 91.42%
Validation Recall: 95.00%
Validation F1 Score: 93.18%


## Conclusion

The accuracy of 92.99% suggests that the model correctly classifies nearly 93% of reviews. The precision of 91.42% implies that most positive predictions are accurate, while the recall of 95.00% shows the model captures almost all actual positive cases.

So we can conclude that based on our research, BERT yeilded the highest accuracy on our Imdb movie review dataset.